<img style="float:left" width="70%" src="pics/escudo_COLOR_1L_DCHA.png">
<img style="float:right" width="15%" src="pics/PythonLogo.svg">
<br style="clear:both;">


# Juego de *Otelo* **inverso**
### Curso 2022-2023



<h2 style="display: inline-block; padding: 4mm; padding-left: 2em; background-color: navy; line-height: 1.3em; color: white; border-radius: 10px;">Propuesta</h2>

## Docentes

 - Pedro Latorre Carmona

## Alumno
- Álvaro Manjón Vara

---

# Reglas y objetivos


El juego de **Otelo** se compone de un tablero con forma cuadrada con 8 casillas de lado ($8\times8 = 64$ casillas en total), y 64 fichas con dos caras: Una, negra y otra, blanca.

En el juego de **Otelo inverso**, para esta **segunda convocatoria**, vamos a tener un tablero de $10\times10$, y el objetivo en este caso es terminar con el **menor número de fichas posible**, teniendo en cuenta que siempre hemos de mover, y que se ha de intentar siempre darle la vuelta a una ficha del jugador contrario.

Por tanto, en esta versión del juego:

- Hay que aumentar el tamaño del tablero, a uno de $10\times10$

- Hay que inventarse una heurística que funcione para el nuevo objetivo del juego.

- Jugadores y dinámica restantes, son iguales.

En esta práctica:

- La división/peso de los puntos a la hora de la calificación es la misma que en el caso de la versión original del juego de **Otelo**.

- Todo lo demás también se mantiene igual, salvo por el objetivo a cumplir y el tamaño del tablero.

In [19]:
from copy import deepcopy
from typing import Tuple, List

# Creación del tablero

In [20]:
def crear_tablero(tamaño: int) -> list:
  """
  Crea un tablero vacío con las fichas iniciales de tamaño n x n.
  """
  # Creación del tablero vacío
  tablero = [[0 for i in range(tamaño)] for j in range(tamaño)]

  # Colocación de las fichas iniciales
  primeraCasilla = int(len(tablero)/2 - 1)
  segundaCasilla = int(len(tablero)/2)
  tablero[primeraCasilla][primeraCasilla] = 1
  tablero[primeraCasilla][segundaCasilla] = 2
  tablero[segundaCasilla][primeraCasilla] = 2
  tablero[segundaCasilla][segundaCasilla] = 1
  
  return tablero

## Creación de la clase TableroOtelo

In [21]:
class TableroOtelo:
  
  def __init__(self, matrix):
    self.setMatrix(matrix)
    

  def __eq__(self, other) -> bool:
    if len(self.matrix) == len(other.matrix):     
      for i in range(self.matrix):
        for j in range(self.matrix):
          if self.matrix[i][j] != other.matrix[i][j]:
            return False
      return True
    return False
      

  def setMatrix(self, matrix):
    self.matrix = deepcopy(matrix)
    

  def getMatrix(self) -> List[List]:
    return deepcopy(self.matrix)
     

  def showMatrix(self):
    for i in range(len(self.matrix)):
      for j in range(len(self.matrix[i])):
        print(str(self.matrix[i][j]) + " ", end='')
      print()


  def placeTile(self, row: int, col: int, tile: int):
    self.matrix[row-1][col-1] = tile
    

  def utility(self) -> int:
    global utilidad
        
    all = [item for sublist in self.matrix for item in sublist]
    white = sum(1 for tile in all if tile == 2)
    black = sum(1 for tile in all if tile == 1)
                
    if white > black:
      diff = int((white / (black + white)) * 100)     
    else:
      diff = - int((black / (black + white)) * 100) 
    
    if self.moveCanBeMade(1) + self.moveCanBeMade(2) == 0:
      mobility = 0    
    else:
      mobility = 100 * self.moveCanBeMade(2) / (self.moveCanBeMade(2) + self.moveCanBeMade(1))
            
    utilidad = diff + mobility
    return utilidad


  def lookForPieces(self, direction: str, firstValue: int, secondValue: int, valRange: int, 
                    playerID: int, element: list, row = 0, col = 0)  -> Tuple[list, bool]:
    changes = []
    searchCompleted = False
    for i in range(firstValue, secondValue, valRange):
      if searchCompleted:
        continue
      piece = element[i]
      if piece == 0:
        changes = []
        searchCompleted = True
      elif piece == playerID:
        searchCompleted = True
      else:
        if direction == "diagonal":
          changes.append((abs(row + (i + 1)), abs(col + (i + 1))))
        else:
          changes.append(i)
    
    return changes, searchCompleted


  def applyChanges(self, direction: str, changes: list, PLAYMODE: bool, value = 0) -> int:
    if PLAYMODE:
      if direction == "horizontal" or direction == "vertical":
        for i in changes:
          if direction == "horizontal":
            self.matrix[value][i] = player
          elif direction == "vertical":
            self.matrix[i][value] = player
      elif direction == "diagonal":
        for i,j in changes:
          self.matrix[i][j] = player
    return len(changes)

    
  def isAvailable(self, row: int, col: int, playerID: int, PLAYMODE:bool) -> int: 
    global changed
    global player
    global debug
    global victory
    global whiteTiles
    global blackTiles

    if PLAYMODE:
      self.matrix[row][col] = player
    count = 0
    __column = self.matrix[row]                        
    __row = [self.matrix[i][col] for i in range(0, len(self.matrix))] 
 
    if playerID in __column[:col]:       
      changes, searchCompleted = self.lookForPieces("horizontal", col - 1, -1, -1, playerID, __column) 
      if searchCompleted:       
        count += self.applyChanges("horizontal", changes, PLAYMODE, row)

    if playerID in __column[col:]:
      changes, searchCompleted = self.lookForPieces("horizontal", col + 1, len(self.matrix), 1, playerID, __column)
      if searchCompleted:       
        count += self.applyChanges("horizontal", changes, PLAYMODE, row)
    
    if playerID in __row[:row]:                         
      changes, searchCompleted = self.lookForPieces("vertical", row - 1, -1, -1, playerID, __row)
      if searchCompleted:
        count += self.applyChanges("vertical", changes, PLAYMODE, col)

    if playerID in __row[row:]:
      changes, searchCompleted = self.lookForPieces("vertical", row + 1, len(self.matrix), 1, playerID, __row)
      if searchCompleted:
        count += self.applyChanges("vertical", changes, PLAYMODE, col)
  
    i = 1    
    ulDiagonal = []
    while row - i >= 0 and col - i >= 0:
      ulDiagonal.append(self.matrix[row - i][col - i])
      i += 1
    if playerID in ulDiagonal:
      changes, searchCompleted = self.lookForPieces("diagonal", 0, len(ulDiagonal), 1, playerID, ulDiagonal, -row, -col)  
      if searchCompleted:
        count += self.applyChanges("diagonal", changes, PLAYMODE)

    i = 1
    urDiagonal = []
    while row + i < len(self.matrix) and col - i >= 0:
      urDiagonal.append(self.matrix[row + i][col - i])
      i += 1
    if playerID in urDiagonal:
      changes, searchCompleted = self.lookForPieces("diagonal", 0, len(urDiagonal), 1, playerID, urDiagonal, row, -col)                                                 
      if searchCompleted:
          count += self.applyChanges("diagonal", changes, PLAYMODE)

    i = 1
    llDiagonal = []
    while row - i >= 0 and col + i < len(self.matrix):
      llDiagonal.append(self.matrix[row - i][col + i])
      i += 1
    if playerID in llDiagonal:
      changes, searchCompleted = self.lookForPieces("diagonal", 0, len(llDiagonal), 1, playerID, llDiagonal, -row, col)                                                                                       
      if searchCompleted:
        count += self.applyChanges("diagonal", changes, PLAYMODE)

    i = 1
    lrDiagonal = []
    while row + i < len(self.matrix) and col + i < len(self.matrix):
      lrDiagonal.append(self.matrix[row + i][col + i])
      i += 1
    if playerID in lrDiagonal:
      changes, searchCompleted = self.lookForPieces("diagonal", 0, len(lrDiagonal), 1, playerID, lrDiagonal, row, col)                                                      
      if searchCompleted:
        count += self.applyChanges("diagonal", changes, PLAYMODE)

    if count == 0 and PLAYMODE:
      self.matrix[row][col] = 0          

    return count
    
    
  def moveCanBeMade(self, playerID:int) -> bool:
    movesFound = False
    for row in range(0, len(self.matrix)):
      for col in range(0, len(self.matrix)):
        if movesFound:
          continue
        elif self.matrix[row][col] == 0:
          numAvailableMoves = self.isAvailable(row, col, playerID, PLAYMODE=False)
          if numAvailableMoves > 0:
            movesFound = True
    
    return movesFound
    
    
  def endGame(self) -> int:
    global victory
    global changed
    global whiteTiles
    global blackTile
    all = [item for sublist in self.matrix for item in sublist]
    white = sum(1 for tile in all if tile == 2)   
    black = sum(1 for tile in all if tile == 1)

    if white > black:
      victory = 2
    elif white < black:
      victory = 1
    else:
      victory = -1
    
    changed = True
    return victory
    
    
  def performMove(self,x:int, y:int):
    global player
    global changed
    if self.matrix[x][y] != 0:   
      print("¡La celda ya está ocupada!")
        
    else:
      numFlipped = self.isAvailable(x, y, player, PLAYMODE=True) 
  
      print("¡Se dieron la vuelta " + str(numFlipped) + " fichas!")
      changed = True
      # check game ending
      allTiles = [item for sublist in self.matrix for item in sublist]
      emptyTiles = sum(1 for tile in allTiles if tile == 0)
      whiteTiles = sum(1 for tile in allTiles if tile == 2)
      blackTiles = sum(1 for tile in allTiles if tile == 1)
  
      print("Estado actual - Vacias: " + str(emptyTiles) + " Blancas: " + str(
        whiteTiles) + " Negras: " + str(blackTiles))
      
      if whiteTiles < 1 or blackTiles < 1 or emptyTiles < 1:
        self.endGame(whiteTiles, blackTiles)
        return
      movesFound = self.moveCanBeMade(3 - player)        
  
  
      if not movesFound:  
        print("¡El jugador " + str(3 - player) + "no puede mover!")
        movesFound = self.moveCanBeMade(player)
        
        if not movesFound: 
          print("¡El jugador " + str(player) + "tampoco puede mover!")
          victoria=self.endGame()
          return victoriaper
        else:
          print("El jugador " + str(player) + " puede mover, asi que ¡adelante!")
          if AIMove(self, player) and player == 2: 
          
            pass   
        
            # AQUÍ es donde llamaremos a la realización del movimiento por parte del ordenador. Por ahora debéis
            # dejarlo comentado. Ésta es la razón por la que he escrito "pass".
            
            # El método correspondiente es "performMoveMinMax"
            
            row, col = performMoveMinMax(self, player)
            self.performMove(row, col)
          
          changed = True
      else:
        player = 3 - player
        changed = True
        
  # Debug, ya veremos si hace falta mantener esto o no
  def startGame(self):
    global player
    player = 1

In [22]:
def miniMax(state:TableroOtelo, currentLevel:int, maxLevel:int, player:int, alpha:int, beta:int, stopDigging:bool) -> Tuple[TableroOtelo, int]:
  matriz=state.getMatrix()
  successorMatrices = []
  
  # No se puede hacer movimiento o hemos llegado al nivel máximo
  if (not state.moveCanBeMade(player) or currentLevel == maxLevel):
    return (state.matrix, state.utility())    
  
  # Exploramos sucesores
  for row in range(0, len(matriz)):
    for col in range(0, len(matriz)):
      if matriz[row][col] == 0:
        numAvailableMoves = state.isAvailable(row, col, player, PLAYMODE=False)
        if numAvailableMoves > 0:
          matrizHija = state.getMatrix()
          matrizHija[row][col] = player
          successorMatrices.append(matrizHija)        
  
  # Si estamos en un nodo hoja
  if len(successorMatrices) == 0:
      stopDigging = True
      coste = state.utility()
      return (state.matrix, coste)
  
  bestMatrix = None  
  if player == 2:  
      minValue = 999 #beta
      for matrix in successorMatrices:
        tableroHijo = TableroOtelo(matrix)
        (matrix, coste) = miniMax(tableroHijo, currentLevel + 1, maxLevel, 1, alpha, beta, stopDigging)
        if coste < minValue:
          minValue = coste
          bestMatrix = matrix
        if minValue <= alpha:
          stopDigging = True
        if minValue < beta:
          beta = minValue
  else:                           
      maxValue = -999 #alpha
      for matrix in successorMatrices:
        tableroHijo = TableroOtelo(matrix)
        (matrix, coste) = miniMax(tableroHijo, currentLevel + 1, maxLevel, 2, alpha, beta, stopDigging)
        if coste > maxValue:
          maxValue = coste
          bestMatrix = matrix
        if maxValue >= beta:
          stopDigging = True
        if maxValue > alpha:
          alpha = maxValue
  
  return (bestMatrix, maxValue)

In [23]:
# Función que debe proponer la casilla (en coordenadas (row,column)) que el ordenador debe ocupar como su siguiente 
# MEJOR MOVIMIENTO

# Tenemos dos alternativas:
    
# 1. Que devuelva la casilla a ocupar, y ya otro programa "pinta" en el tablero tanto la ocupación de la casilla, como la 
# actualización
# 2. Que dentro del programa llame a otro programa, si éste es el que "pinta" la nueva versión del tablero
def performMoveMinMax(state:TableroOtelo, player:int):
    matrizResultado, coste = miniMax(state, 0, 1, player, -999, 999, False)
    for row in range(0, len(matrizResultado)):
      for col in range(0, len(matrizResultado[row])):
        if matrizResultado[row][col] != state.getMatrix()[row][col]:
          movimiento = (row, col)
          return movimiento

In [24]:
# Función que llama a "performMoveMinMax" en el caso directamente sea la parte de AI la que tiene que mover, y aparte
# de devolver el movimiento, utiliza una variable "booleana" llamada "AIReadyToMove" que determinará, de cara al resto
# del código, si le corresponde o no mover al ordenador.

def AIMove(state:TableroOtelo, player:int):
    global AIReadyToMove

    AIReadyToMove = state.moveCanBeMade(player)

    return AIReadyToMove
   

In [34]:
tablero = TableroOtelo(crear_tablero(10))
tablero.startGame()
tablero.performMove(3,5)
tablero.showMatrix()
row, col = performMoveMinMax(tablero, 2)
tablero.performMove(row, col)
tablero.showMatrix()
tablero.performMove(3,3)
tablero.showMatrix()
row, col = performMoveMinMax(tablero, 2)
tablero.performMove(row, col)
tablero.showMatrix()

¡Se dieron la vuelta 1 fichas!
Estado actual - Vacias: 95 Blancas: 1 Negras: 4
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 1 0 0 0 0 
0 0 0 0 1 1 0 0 0 0 
0 0 0 0 2 1 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
¡Se dieron la vuelta 1 fichas!
Estado actual - Vacias: 94 Blancas: 3 Negras: 3
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 2 1 0 0 0 0 
0 0 0 0 2 1 0 0 0 0 
0 0 0 0 2 1 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
¡Se dieron la vuelta 2 fichas!
Estado actual - Vacias: 93 Blancas: 1 Negras: 6
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 1 1 1 0 0 0 0 
0 0 0 0 1 1 0 0 0 0 
0 0 0 0 2 1 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
¡Se dieron la vuelta 2 fichas!
Estado actual - Vacias: 92 Blancas: 4 Negras: 4
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 2 0 